In [18]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model
import matplotlib.pyplot as plt



df = pd.read_excel(r'Copy of Extraction PT4 14x59 din 2016 pana in prezent - analysis S1928 (003) (3).xlsx', sheet_name='TL4 -40 Nm' )
dataset = df.values
X = dataset[1:,23:42]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')


d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,3,5,7,8,9,10,11,13,15,17], axis=1, inplace=True)
d2.replace(0.  , np.nan, inplace=True)
d2.replace(-1. , np.nan, inplace=True)
d2.replace(1.  , np.nan, inplace=True)
d2.replace(2.  , np.nan, inplace=True)
d2.replace(3.  , np.nan, inplace=True)
d2.replace(4.  , np.nan, inplace=True)
d2.replace(5.  , np.nan, inplace=True)
d2.replace(6.  , np.nan, inplace=True)

d1[7] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)




X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape




def basic_model_3(x):
     # create model
    model = Sequential()
    model.add(Dense(x, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(x, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(x, input_dim=2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def error(x):
    emax = 1.
    emin = 1.
    for i in range(x):
        p = model.predict(X_train[i].reshape(1, 8))
        #print(p, Y_train[i], p/float(Y_train[i]))
        pp = Y_scaler.inverse_transform(p)
        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
        r = p/float(Y_train[i])
        if r > emax and py > 7.:
            emax = r
        if r < emin and py > 7.:
            emin = r
#        print(pp,py, r)
    return emin , emax;

rmin = 10
rmax = 11

for x4 in range(rmin,rmax):
                dif=3.0
                model = basic_model_3(x4)
                epochs = 10000
                batch_size = 128
                #print('Epochs: ', epochs)
                #print('Batch size: ', batch_size)
                keras_callbacks = [
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
                # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
                EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
                ]



                history = model.fit(X_train, Y_train,
                batch_size=batch_size,
                epochs=epochs,
                shuffle=True,
                verbose=0, # Change it to 2, if wished to observe execution
                validation_data=(X_test, Y_test),
                callbacks=keras_callbacks)
                
                train_score = model.evaluate(X_train, Y_train, verbose=0)
                valid_score = model.evaluate(X_test, Y_test, verbose=0)
            
                emn , ema = error(50)
                print('errmin,errmax,x4:',emn,ema , x4)
               

C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


errmin,errmax,x4: [[0.32819068]] [[1.5206164]] 10


In [38]:
def tree_model(x, dropout=0.2):
    model = Sequential()
    model.add(Dense(x, input_dim=8, kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/4),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


def err(model, x):
    emax = 1.
    emin = 1.
    for i in range(x):
        p = model.predict(X_train[i].reshape(1, 8))
        #print(p, Y_train[i], p/float(Y_train[i]))
        pp = Y_scaler.inverse_transform(p)
        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
        r = p/float(Y_train[i])
        if r > emax and py > 7.:
            emax = r
        if r < emin and py > 7.:
            emin = r
#        print(pp,py, r)
    return emin , emax;

def process_model(model, batch_size=32, epochs=500):
    history = model.fit(X_train, Y_train, batch_size=batch_size,epochs=epochs,
                shuffle=True, verbose=0, validation_data=(X_test, Y_test), callbacks=keras_callbacks)
                
    train_score = model.evaluate(X_train, Y_train, verbose=0)
    valid_score = model.evaluate(X_test, Y_test, verbose=0)
    
model = tree_model(10)    
process_model(model, batch_size=32, epochs=500)

emn , ema = err(model, 50)
print('errmin,errmax,x4:',emn,ema , x4)


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


errmin,errmax,x4: [[0.5777077]] [[1.4983128]] 10


In [32]:
m1 = tree_model(8)
m2 = basic_model_3(8)
process_model(m1)
process_model(m2)


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [33]:
emn , ema = err(m1, 50)
print('errmin,errmax,x4:',emn,ema , x4)

errmin,errmax,x4: [[0.5770872]] [[1.4967035]] 10


In [34]:
emn , ema = err(m2, 50)
print('errmin,errmax,x4:',emn,ema , x4)

errmin,errmax,x4: [[0.56217074]] [[1.5265647]] 10


In [43]:
# Am selectat modelul tree de 8 biti ca modelul cel mai potrivit
# verificam posibilitatea de a minimiza abaterea folosind dropout
for i in [None, 0.1, 0.2, 0.3, 0.4]:
    m1 = tree_model(8, i)
    process_model(m1)
    print(err(m1,50),i)

C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.5757442]], dtype=float32), array([[1.5847133]], dtype=float32)) None


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.5690359]], dtype=float32), array([[1.4849854]], dtype=float32)) 0.1


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.5493212]], dtype=float32), array([[1.4575806]], dtype=float32)) 0.2


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.54119664]], dtype=float32), array([[1.452147]], dtype=float32)) 0.3


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.5757576]], dtype=float32), array([[1.4891092]], dtype=float32)) 0.4


In [47]:
mm = tree_model(8,dropout=0.2)
bs = 64
for epoch in [10000,5000,2500,1000,500]:
    process_model(mm, batch_size=bs, epochs=epoch)
    print(err(mm, 50), bs)

C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.57994586]], dtype=float32), array([[1.5041176]], dtype=float32)) 64


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.5789465]], dtype=float32), array([[1.5015256]], dtype=float32)) 64


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.58032185]], dtype=float32), array([[1.5050929]], dtype=float32)) 64


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.57998395]], dtype=float32), array([[1.5042164]], dtype=float32)) 64


C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.5806997]], dtype=float32), array([[1.5060728]], dtype=float32)) 64


In [50]:
mm.save('64T8-02.h5p')

In [57]:

def etree_model(x, dropout=0.2):
    model = Sequential()
    model.add(Dense(x, input_dim=8, kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x*2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/4),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

em = etree_model(8, dropout=0.2)
process_model(em, batch_size=64, epochs=5000)
print(err(em, 50), bs)

C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.5501057]], dtype=float32), array([[1.4152333]], dtype=float32)) 64
